# Tag 1: Einführung & Grundlagen
## Unitree G1 Humanoid Roboter Academy

**Willkommen zur EF Robotics Unitree G1 Academy!**

### Agenda Tag 1:
1. Begrüßung & Vorstellung EF Robotics
2. Übersicht Unitree G1 Humanoid Roboter
3. Hardware Architektur
4. Software Komponenten
5. Setup & Inbetriebnahme
6. Safety-Aspekte
7. Praktische Übungen

---
## 1. Übersicht Unitree G1

Der **Unitree G1** ist ein hochmoderner humanoider Roboter mit folgenden Hauptmerkmalen:

### Hauptspezifikationen:
- **DOF (Degrees of Freedom)**: 
  - 23 DOF (Basis-Konfiguration ohne Arme)
  - 29 DOF (mit Dual-Arms)
  - 49 DOF (mit Dual-Arms + 20-DOF Händen)
- **Höhe**: ~130 cm
- **Gewicht**: ~35 kg
- **Maximale Geschwindigkeit**: 0.6 m/s (linear)
- **Akkulaufzeit**: ~2-4 Stunden (abhängig von der Last)

### Anwendungsbereiche:
- Forschung in Humanoid-Robotik
- Autonome Navigation in menschlichen Umgebungen
- Manipulation und Objektinteraktion
- Reinforcement Learning für komplexe Bewegungsaufgaben

---
## 2. Hardware Architektur

### 2.1 Mechanischer Aufbau

```
┌─────────────────────┐
│   Kopf (Head)       │  ← Sensoren (RealSense, IMU)
├─────────────────────┤
│   Torso/Pelvis      │  ← Hauptcomputer (Jetson)
│   ┌───┐     ┌───┐   │
│   │ L │     │ R │   │  ← Dual Arms (je 7 DOF)
│   │ Arm│     │Arm│   │
│   └───┘     └───┘   │
├──────┬───┬──────────┤
│  L   │Hip│   R      │
│  Leg │   │   Leg    │  ← Beine (je 6 DOF)
│      │   │          │
└──────┴───┴──────────┘
```

### 2.2 Gelenkstruktur

**Beine (12 DOF total):**
- Hip Yaw/Roll/Pitch (3 DOF)
- Knee Pitch (1 DOF)
- Ankle Pitch/Roll (2 DOF)

**Arme (14 DOF total):**
- Shoulder Pitch/Roll/Yaw (3 DOF)
- Elbow Pitch (1 DOF)
- Wrist Yaw/Roll/Pitch (3 DOF)

**Torso:**
- Waist Yaw (1 DOF)

**Hände (optional, 20 DOF total):**
- 5 Finger mit je 4 Gelenken

### 2.3 Sensorausstattung

| Sensor | Typ | Funktion |
|--------|-----|----------|
| **IMU** | 6-Achsen (Gyro + Accel) | Lage- und Beschleunigungsmessung |
| **RealSense D435i** | RGB-D Kamera | Tiefenwahrnehmung, Objekterkennung |
| **Livox MID-360** | 3D LiDAR | 360° Umgebungsscan, SLAM |
| **Fußkraftsensoren** | Kraftsensoren | Bodenkontakterkennung |
| **Gelenk-Encoder** | Magnetische Encoder | Positionsfeedback aller Motoren |

---
## 3. Software Architektur

### 3.1 Software-Stack Übersicht

```
┌──────────────────────────────────────────┐
│  Benutzer-Anwendungen & GUI              │
│  (run_geoff_gui.py, keyboard_controller) │
├──────────────────────────────────────────┤
│  High-Level APIs                         │
│  - LocoClient (Bewegungssteuerung)       │
│  - SLAM (KISS-ICP)                       │
│  - Perception (RealSense, LiDAR)         │
├──────────────────────────────────────────┤
│  Middleware                              │
│  - Unitree SDK-2 (DDS)                   │
│  - GStreamer (Video-Streaming)           │
├──────────────────────────────────────────┤
│  Low-Level                               │
│  - Motor Controller                      │
│  - Sensor Drivers                        │
└──────────────────────────────────────────┘
```

### 3.2 Finite State Machine (FSM)

Der G1 verwendet eine **FSM** zur Zustandsverwaltung:

| FSM ID | Name | Beschreibung |
|--------|------|-------------|
| **0** | Zero Torque | Motoren aus (Notfall) |
| **1** | Damp | Weiche Gelenke (Startup-Zustand) |
| **2** | Squat | Hockstellung |
| **4** | StandUp Helper | Aufstehhilfe aus Hängung |
| **200** | Start | **Balance-Controller aktiv, Gehen möglich** |
| **702** | Lie to Stand | Aufstehen aus Liegeposition |
| **706** | Squat to Stand | Aufstehen aus Hockstellung |

### 3.3 Threading-Modell

Die Software nutzt **Lock-Protected Shared State** ohne ROS:

```python
# Beispiel: Multi-Threading Architektur
Thread 1: RealSense RX  → shared_rgb_frame (lock)
Thread 2: SLAM          → shared_pose (lock)
Thread 3: Robot Control → shared_cmd_vel (lock)
Thread 4: GUI Render    → liest alle shared states
```

**Vorteile:**
- Kein ROS-Overhead
- Einfaches Debugging
- Geringer Latenz

---
## 4. Setup & Inbetriebnahme

### 4.1 Systemvoraussetzungen

**Hardware:**
- Unitree G1 Roboter
- Laptop/PC mit Ubuntu 20.04/22.04
- Netzwerkverbindung (Ethernet empfohlen)

**Software:**
- Python 3.8+
- Unitree SDK-2
- GStreamer
- Abhängigkeiten (siehe requirements.txt)

### 4.2 Installation

**Schritt 1: Repository klonen**

In [ ]:
# Im Terminal ausführen:
# cd ~/Desktop/EF/unitree
# git clone https://github.com/your-repo/unitree_g1_vibes.git
# cd unitree_g1_vibes

**Schritt 2: Python-Abhängigkeiten installieren**

In [ ]:
# Im Terminal:
# pip install -r requirements.txt
# pip install pyside6 pyqtgraph pynput stable-baselines3

**Schritt 3: Unitree SDK-2 installieren**

In [ ]:
# Im Terminal:
# pip install unitree_sdk2py

### 4.3 Netzwerk-Konfiguration

**Wichtig:** Der G1 kommuniziert über **DDS (Data Distribution Service)** via Ethernet.

1. Verbinden Sie Ihren PC mit dem G1 über Ethernet
2. Finden Sie Ihr Netzwerk-Interface:

In [ ]:
# Im Terminal:
# ip addr show
# Beispiel-Output: enp68s0f1 oder eth0

3. Merken Sie sich den Interface-Namen (z.B. `enp68s0f1`)

---
## 5. Praktische Übung 1: Hanger Boot Sequence

### Ziel:
Den Roboter aus dem Hängezustand in den laufbereiten Zustand bringen.

### Konzept:
Der G1 wird typischerweise in einer **Hängung** gelagert (Füße berühren den Boden nicht). Beim Start:
1. Roboter ist im `Damp`-Modus (FSM 1)
2. Schrittweise Stand-Höhe erhöhen
3. Warten bis Füße belastet sind
4. Balance-Controller aktivieren (FSM 200)

In [ ]:
# Importieren Sie die notwendigen Bibliotheken
import sys
sys.path.append('../unitree_g1_vibes')

from hanger_boot_sequence import hanger_boot_with_retry

# WICHTIG: Ersetzen Sie 'enp68s0f1' mit Ihrem Netzwerk-Interface!
NETWORK_INTERFACE = 'enp68s0f1'  # Beispiel, anpassen!

print("Starte Hanger Boot Sequence...")
print("WARNUNG: Stellen Sie sicher, dass der Roboter sicher hängt!")
print("Drücken Sie Ctrl+C zum Abbrechen\n")

In [ ]:
# Hanger Boot Sequence ausführen
# ACHTUNG: Nur ausführen wenn Roboter sicher hängt!

# Unkommentieren zum Ausführen:
# bot = hanger_boot_with_retry(
#     network_interface=NETWORK_INTERFACE,
#     step_size=0.02,  # 2cm pro Schritt
#     max_height=0.5   # Maximale Höhe 50cm
# )
#
# if bot:
#     print("✓ Boot erfolgreich! Roboter bereit für Move()-Befehle")
#     print(f"Aktueller FSM: {bot.state.sport_mode_state.mode}")
# else:
#     print("✗ Boot fehlgeschlagen")

### Was passiert intern?

```python
# Vereinfachte Version des Boot-Prozesses:
# 1. Damp-Modus aktivieren
bot.Damp()
time.sleep(0.5)

# 2. StandUp Helper aktivieren
bot.SetFsmId(4)
time.sleep(0.5)

# 3. Schrittweise Stand-Höhe erhöhen
current_height = 0.0
while current_height < max_height:
    bot.SetStandHeight(current_height)
    current_height += step_size
    time.sleep(0.1)
    
    # Prüfen ob Füße belastet sind
    if bot.state.sport_mode_state.mode == 0:
        break  # Füße haben Bodenkontakt!

# 4. Balance-Controller aktivieren
bot.BalanceStand(0)
time.sleep(0.5)

# 5. In FSM 200 wechseln (Start)
bot.Start()
```

---
## 6. Praktische Übung 2: Erste Bewegungsbefehle

### Grundlegende Bewegungs-API

Nach erfolgreichem Boot (FSM 200) können Sie den Roboter steuern:

In [ ]:
# Beispiel: Einfache Bewegungen
# (Nur ausführen wenn Roboter im FSM 200!)

# Unkommentieren zum Testen:
# import time
#
# # 1. Vorwärtsgehen (0.2 m/s)
# print("Gehe vorwärts...")
# bot.Move(vx=0.2, vy=0.0, omega=0.0)
# time.sleep(3)
#
# # 2. Stoppen
# print("Stoppe...")
# bot.Move(vx=0.0, vy=0.0, omega=0.0)
# time.sleep(1)
#
# # 3. Seitwärtsgehen (links)
# print("Gehe seitwärts...")
# bot.Move(vx=0.0, vy=0.1, omega=0.0)
# time.sleep(3)
#
# # 4. Rotation (Drehung um eigene Achse)
# print("Drehe...")
# bot.Move(vx=0.0, vy=0.0, omega=0.3)  # rad/s
# time.sleep(3)
#
# # 5. Finale Stopp
# bot.StopMove()
# print("Bewegungen abgeschlossen!")

### Move() API Parameter:

```python
bot.Move(vx, vy, omega, continuous_move=False)
```

| Parameter | Typ | Einheit | Beschreibung |
|-----------|-----|---------|-------------|
| `vx` | float | m/s | Vorwärts/Rückwärts-Geschwindigkeit (-0.6 bis +0.6) |
| `vy` | float | m/s | Seitwärts-Geschwindigkeit (-0.6 bis +0.6) |
| `omega` | float | rad/s | Rotationsgeschwindigkeit (-0.6 bis +0.6) |
| `continuous_move` | bool | - | True = Kontinuierliche Updates, False = Einmaliger Befehl |

---
## 7. Safety-Aspekte

### 7.1 Grundlegende Sicherheitsregeln

**VOR dem Start:**
1. ✓ Arbeitsbereich frei von Hindernissen (mind. 3m Radius)
2. ✓ Notfall-Stopp-Prozedur bekannt
3. ✓ Batterie ausreichend geladen (>30%)
4. ✓ Netzwerkverbindung stabil

**WÄHREND des Betriebs:**
1. ⚠ Nie unter/über den Roboter greifen während er sich bewegt
2. ⚠ Geschwindigkeiten begrenzen (max 0.6 m/s)
3. ⚠ Kontinuierlich Zustand überwachen
4. ⚠ Bei unerwartetem Verhalten → Sofort stoppen

**Notfall-Stopp:**
```python
# Methode 1: Sanfter Stopp
bot.StopMove()

# Methode 2: Damp (Weiche Gelenke)
bot.Damp()

# Methode 3: Zero Torque (Notfall!)
bot.ZeroTorque()
```

### 7.2 Automatische Sicherheitsmechanismen

Der G1 hat **eingebaute Safety-Features:**

1. **Geschwindigkeitsbegrenzung**: Automatisches Clamping auf max. 0.6 m/s
2. **Fußkraft-Monitoring**: Erkennt fehlenden Bodenkontakt
3. **IMU-Überwachung**: Erkennt Kippen/Fallen
4. **Watchdog**: Automatischer Stopp bei Kommunikationsverlust
5. **Batterie-Warnung**: Alert bei <20% Ladung

### 7.3 Übung: Zustandsüberwachung

In [ ]:
# Roboter-Zustand auslesen
# Unkommentieren zum Testen:

# if 'bot' in locals():
#     print("=== Roboter-Status ===")
#     print(f"FSM Modus: {bot.state.sport_mode_state.mode}")
#     print(f"Position: {bot.state.sport_mode_state.position}")
#     print(f"Geschwindigkeit: {bot.state.sport_mode_state.velocity}")
#     print(f"Fußkraft Links: {bot.state.sport_mode_state.foot_force[0]}")
#     print(f"Fußkraft Rechts: {bot.state.sport_mode_state.foot_force[1]}")
#     print(f"IMU Orientierung: {bot.state.imu_state.quaternion}")
# else:
#     print("Roboter nicht verbunden. Führen Sie zuerst die Hanger Boot Sequence aus.")

---
## 8. Zusammenfassung Tag 1

### Gelernte Konzepte:
✓ Hardware-Architektur des G1 (DOF, Sensoren, Aktuatoren)  
✓ Software-Stack (FSM, Threading, SDK)  
✓ Setup & Netzwerk-Konfiguration  
✓ Hanger Boot Sequence  
✓ Grundlegende Bewegungsbefehle (Move API)  
✓ Safety-Protokolle  

### Wichtige Dateien:
- `hanger_boot_sequence.py` - Startup-Automatisierung
- `g1_loco_client_example.py` - Bewegungs-Beispiele
- `FSM_README.md` - FSM-Referenz
- `docs/quick_start.md` - Schnellstart-Anleitung

---
## 9. Hausaufgabe / Vertiefung

**Aufgabe 1:** Lesen Sie die FSM-Dokumentation (`FSM_README.md`) und erstellen Sie eine Übersicht aller FSM-Zustände.

**Aufgabe 2:** Experimentieren Sie mit verschiedenen Geschwindigkeitskombinationen:
- Diagonales Gehen (vx + vy gleichzeitig)
- Kreisfahrt (vx + omega)
- Seitwärts mit Rotation

**Aufgabe 3:** Implementieren Sie eine Funktion `safe_move()`, die:
1. Geschwindigkeiten automatisch begrenzt
2. Fußkraft überwacht
3. Bei Problemen automatisch stoppt

**Bonusaufgabe:** Erstellen Sie ein simples Terminal-Interface für manuelle Bewegungssteuerung.

---
## 10. Nächster Tag: Bewegungssteuerung & Laufverhalten

**Vorschau Tag 2:**
- Dynamisches Gehen und Stabilität
- Regelungstechnik für Humanoiden
- Reinforcement Learning Ansätze
- Workshop: Keyboard-Tele-Operation
- Praktische Trajektorien-Planung